In [2]:
import math
import os
import datetime

import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model, Sequential

from keras.utils import to_categorical
from keras.optimizers import RMSprop, Adam

from keras.applications import xception
from keras.applications import inception_v3

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

from secrets import KAGGLE_USER, KAGGLE_PW

Using TensorFlow backend.


In [3]:
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/preprocessed'

gen = image.ImageDataGenerator()

batch_size = 16
target_size=(299, 299)

def add_preprocess(base_model, preprocess_func, inputs_shape=(299, 299, 3)):
    inputs = Input(shape=inputs_shape)
    x = Lambda(preprocess_func)(inputs)
    outputs = base_model(x)
    model = Model(inputs, outputs)
    return model

In [4]:
batches = gen.flow_from_directory(data_dir+'/train', shuffle=False, target_size=target_size, batch_size=batch_size)
batches_val = gen.flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)

nb_batches = math.ceil(batches.n/batch_size)
nb_batches_val = math.ceil(batches_val.n/batch_size)

y_encode = batches.classes
y_val_encode = batches_val.classes

y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)

Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.


In [5]:
base_model = xception.Xception(weights='imagenet', include_top=False, pooling='avg')

model_x = add_preprocess(base_model, xception.preprocess_input)

# bf_x=model_x.predict_generator(batches, steps=nb_batches, verbose=1)
# np.save(data_dir+'/results/bf_x', bf_x)
bf_x = np.load(data_dir+'/results/bf_x.npy')
# bf_val_x=model_x.predict_generator(batches_val, steps=nb_batches_val, verbose=1)
# np.save(data_dir+'/results/bf_val_x', bf_val_x)
bf_val_x = np.load(data_dir+'/results/bf_val_x.npy')

In [7]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logreg.fit(bf_x, y_encode)
valid_probs = logreg.predict_proba(bf_val_x)
valid_preds = logreg.predict(bf_val_x)
print('logloss:', log_loss(y_val_encode, valid_probs))
print('accuracy:', accuracy_score(y_val_encode, valid_preds))

logloss: 0.325739933423
accuracy: 0.904


In [9]:
lm = Sequential([Dense(120, activation='softmax', input_shape=(2048,))])
lm.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
lm.fit(bf_x, y, epochs=15, batch_size=nb_batches, validation_data=(bf_val_x, y_val))

Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 0s - loss: 3.0704 - acc: 0.6703 - val_loss: 1.9152 - val_acc: 0.8495
Epoch 2/15
8222/8222 [==============================] - 0s - loss: 1.3568 - acc: 0.8852 - val_loss: 0.9857 - val_acc: 0.8805
Epoch 3/15
8222/8222 [==============================] - 0s - loss: 0.7460 - acc: 0.9032 - val_loss: 0.6382 - val_acc: 0.8935
Epoch 4/15
8222/8222 [==============================] - 0s - loss: 0.5036 - acc: 0.9117 - val_loss: 0.4865 - val_acc: 0.9015
Epoch 5/15
8222/8222 [==============================] - 0s - loss: 0.3875 - acc: 0.9191 - val_loss: 0.4089 - val_acc: 0.9005
Epoch 6/15
8222/8222 [==============================] - 0s - loss: 0.3212 - acc: 0.9241 - val_loss: 0.3656 - val_acc: 0.9050
Epoch 7/15
8222/8222 [==============================] - 0s - loss: 0.2782 - acc: 0.9306 - val_loss: 0.3438 - val_acc: 0.9075
Epoch 8/15
8222/8222 [==============================] - 0s - loss: 0.2471 - a

In [15]:
lm = Sequential([Dense(120, activation='softmax', input_shape=(2048,))])
lm.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
lm.fit(bf_x, y, epochs=50, batch_size=nb_batches, validation_data=(bf_val_x, y_val))

Train on 8222 samples, validate on 2000 samples
Epoch 1/50
8222/8222 [==============================] - 0s - loss: 3.8639 - acc: 0.4704 - val_loss: 2.7769 - val_acc: 0.8120
Epoch 2/50
8222/8222 [==============================] - 0s - loss: 1.9589 - acc: 0.8621 - val_loss: 1.3256 - val_acc: 0.8745
Epoch 3/50
8222/8222 [==============================] - 0s - loss: 0.9387 - acc: 0.8948 - val_loss: 0.7487 - val_acc: 0.8930
Epoch 4/50
8222/8222 [==============================] - 0s - loss: 0.5832 - acc: 0.9091 - val_loss: 0.5552 - val_acc: 0.8965
Epoch 5/50
8222/8222 [==============================] - 0s - loss: 0.4530 - acc: 0.9160 - val_loss: 0.4744 - val_acc: 0.9015
Epoch 6/50
8222/8222 [==============================] - 0s - loss: 0.3880 - acc: 0.9203 - val_loss: 0.4305 - val_acc: 0.9045
Epoch 7/50
8222/8222 [==============================] - 0s - loss: 0.3465 - acc: 0.9264 - val_loss: 0.4015 - val_acc: 0.9055
Epoch 8/50
8222/8222 [==============================] - 0s - loss: 0.3165 - a

In [17]:
lm = Sequential([Dense(120, activation='softmax', input_shape=(2048,))])
lm.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
lm.fit(bf_x, y, epochs=50, batch_size=nb_batches, validation_data=(bf_val_x, y_val))
lm.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
lm.fit(bf_x, y, epochs=5, batch_size=nb_batches, validation_data=(bf_val_x, y_val))

Train on 8222 samples, validate on 2000 samples
Epoch 1/50
8222/8222 [==============================] - 0s - loss: 3.8434 - acc: 0.4696 - val_loss: 2.7352 - val_acc: 0.8220
Epoch 2/50
8222/8222 [==============================] - 0s - loss: 1.9314 - acc: 0.8667 - val_loss: 1.2930 - val_acc: 0.8820
Epoch 3/50
8222/8222 [==============================] - 0s - loss: 0.9219 - acc: 0.8999 - val_loss: 0.7350 - val_acc: 0.8935
Epoch 4/50
8222/8222 [==============================] - 0s - loss: 0.5759 - acc: 0.9108 - val_loss: 0.5445 - val_acc: 0.9025
Epoch 5/50
8222/8222 [==============================] - 0s - loss: 0.4483 - acc: 0.9158 - val_loss: 0.4674 - val_acc: 0.9040
Epoch 6/50
8222/8222 [==============================] - 0s - loss: 0.3846 - acc: 0.9211 - val_loss: 0.4267 - val_acc: 0.9000
Epoch 7/50
8222/8222 [==============================] - 0s - loss: 0.3442 - acc: 0.9256 - val_loss: 0.3978 - val_acc: 0.9085
Epoch 8/50
8222/8222 [==============================] - 0s - loss: 0.3145 - a